In [5]:
import numpy as np
import scipy.stats

probably want to change this to class structure

In [ ]:
class Contract:
    def __init__(self):
        self.KPI_goal = 0
        self.T = 0
        self.observations = 0
        self.maturity_underlying = 0
        self.bps = 0
        self.principal = 0

In [ ]:
class Dynamics:
    def __init__(self):
        self.KPI_0 = 0.0
        self.r = 0.0
        self.sigma = 0.0

In [ ]:
class Tree: 
    def __init__(self):
        self.N = 0

insert example contract and dynamics here

# Black-Scholes Method

AONC European Call (but in reality its closer to a american?)

In [6]:
def BS_call(dynamics,contract):
    KPI_0,sigma,r = dynamics.KPI_0,dynamics.sigma,dynamics.r
    KPI_goal,T,maturity_underlying = contract.KPI_goal,contract.T,contract.maturity_underlying
    observations,bps,P = contract.observations,contract.bps,contract.P
    d2 = (np.log(KPI_0/KPI_goal)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
    Q = bps/10000*P*(maturity_underlying-T)*observations
    return Q*np.exp(-r*T)*scipy.stats.norm(d2)

# Binomial Tree Method

In [1]:
def bin_tree_call(KPI_0,KPI_goal,sigma,r,T,N,observations,maturity_underlying,bps,P): 
    delta_t = T/N 
    discount_factor = np.exp(-r*delta_t)
    u = np.exp(sigma*np.sqrt(delta_t))
    d = 1/u
    p = (np.exp(r*delta_t)-d)/(u-d)
    #constructing stock tree
    KPI = np.zeros(N+1,N+1)
    for time in range(N+1):
        for node in range(time):
            KPI[time][node] = KPI_0*(u**node)*(d**(time-node))
    payoff = np.zeros(N+1,N+1)
    for i in range(N+1):
        if KPI[i][N] >= KPI_goal:
            payoff[i][N] = bps/10000*P
        else:
            payoff[i][N] = 0
    discount_payoff = 0
    for i in range(N+1):
        discount_payoff += payoff[i][N]*(math.factorial(N)/(math.factorial(i)*math.factorial(N-i)*(p**i)*((1-p)**(time-i)))
    
            
        

SyntaxError: unexpected EOF while parsing (4201832125.py, line 23)

# Paper Implementation

In [ ]:
def paper_bin_tree(KPI_0,KPI_u,KPI_goal,sigma,r,T,N,observations,maturity_underlying,bps,P):
    delta_t = T/N 
    discount_factor = np.exp(-r*delta_t)
    u = 1+(KPI_u - KPI_0)/(KPI_0)
    d = 1/u
    sigma = np.log(u)/np.sqrt(delta_t)
    p = (np.exp(r*delta_t)-d)/(u-d)
    KPI = np.zeros(N+1)
    KPI[0] = KPI_0*d**N
    #construct the final set of prices at T
    for n in range(1,N+1):
        KPI[n] = KPI[n-1]*u/d
    #option value at maturity
    payoff = np.zeros(N+1)
    for n in range(N+1):
        if KPI[n] >= KPI_goal:
            payoff[n] = bps/10000*P
        else:
            payoff[n] = 0
    # 
    for time in np.range(N,0,-1):
        for node in range(0,time):
            payoff[node] = discount_factor * p*payoff[node+1]+ (1-p)*payoff[node]
    return payoff[0] * (maturity_underlying-T)*observations

            